In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_16207/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [ ]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

In [8]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb


In [9]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'pb']]}
)

In [10]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[14:43:33] Stdout logging level is INFO2.
[14:43:33] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:43:33] Task: multilabel

[14:43:33] Start automl preset with listed constraints:
[14:43:33] - time: 3600.00 seconds
[14:43:33] - CPU: 1 cores
[14:43:33] - memory: 16 GB

[14:43:33] Train data shape: (14644, 22)
[14:43:34] Feats was rejected during automatic roles guess: []
[14:43:34] Layer 1 train process start. Time left 3599.40 secs
[14:43:34] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:43:34] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:43:38] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:43:41] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:43:45] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.376073239155966
[14:43:45] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:43:45] Time left 3587.64 s

In [11]:
gpu_inf = automl_gpu.predict(data)

In [12]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[14:44:15] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[14:44:15] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[14:44:15] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[14:44:15] CatBoost uses as obj. MultiCrossEntropy.


  0%|          | 0/149 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[14:44:17] CatBoost uses as obj. MultiCrossEntropy.


In [13]:
cpu_inf = automl_gpu.predict(data)

In [14]:
from joblib import dump, load
import time
pickle_file = './gpu.joblib'

In [15]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_gpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.140s


In [16]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


Raw load duration: 0.097s


In [17]:
model_inf.levels[0][1].ml_algos[0].models[0].__dict__

{'tl_model': <treelite.frontend.Model at 0x7f03f7da1c10>,
 'postprocess_fn': <lightautoml_gpu.tasks.losses.bce_loss.BCELossCPU at 0x7f03f7da1cd0>}

In [18]:
disk_pred = model_inf.predict(data)

In [19]:
disk_pred.data.T

array([[0.03836581, 0.07403237, 0.04497298, ..., 0.2328304 , 0.03760371,
        0.04118644],
       [0.03474031, 0.06981577, 0.03458579, ..., 0.05481545, 0.02665372,
        0.0533637 ],
       [0.01229703, 0.01670735, 0.01232157, ..., 0.00294785, 0.01023321,
        0.01007054],
       ...,
       [0.00416759, 0.01176112, 0.00473114, ..., 0.0086079 , 0.00555177,
        0.004463  ],
       [0.00141493, 0.0025898 , 0.00150979, ..., 0.00515158, 0.0018137 ,
        0.00159866],
       [0.10493021, 0.1681763 , 0.11440004, ..., 0.25396088, 0.10606527,
        0.13573591]], dtype=float32)

In [20]:
disk_pred.data.T

array([[0.03836581, 0.07403237, 0.04497298, ..., 0.2328304 , 0.03760371,
        0.04118644],
       [0.03474031, 0.06981577, 0.03458579, ..., 0.05481545, 0.02665372,
        0.0533637 ],
       [0.01229703, 0.01670735, 0.01232157, ..., 0.00294785, 0.01023321,
        0.01007054],
       ...,
       [0.00416759, 0.01176112, 0.00473114, ..., 0.0086079 , 0.00555177,
        0.004463  ],
       [0.00141493, 0.0025898 , 0.00150979, ..., 0.00515158, 0.0018137 ,
        0.00159866],
       [0.10493021, 0.1681763 , 0.11440004, ..., 0.25396088, 0.10606527,
        0.13573591]], dtype=float32)

In [21]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
#print(oof_pred.data.T)
print()
print(disk_pred.data.T)

[[0.03836581 0.07403237 0.04497298 ... 0.2328304  0.03760371 0.04118644]
 [0.03474031 0.06981577 0.03458579 ... 0.05481545 0.02665372 0.0533637 ]
 [0.01229703 0.01670735 0.01232157 ... 0.00294785 0.01023321 0.01007054]
 ...
 [0.00416759 0.01176112 0.00473114 ... 0.0086079  0.00555177 0.004463  ]
 [0.00141493 0.0025898  0.00150979 ... 0.00515158 0.0018137  0.00159866]
 [0.10493021 0.1681763  0.11440004 ... 0.25396088 0.10606527 0.13573591]]

[[0.0383658  0.07403239 0.044973   ... 0.2328304  0.03760373 0.04118645]
 [0.03474031 0.06981574 0.03458579 ... 0.05481544 0.02665372 0.05336367]
 [0.01229703 0.01670738 0.01232156 ... 0.00294785 0.01023321 0.01007054]
 ...
 [0.00416759 0.01176112 0.00473114 ... 0.00860791 0.00555177 0.004463  ]
 [0.00141493 0.0025898  0.00150979 ... 0.00515158 0.00181371 0.00159866]
 [0.10493021 0.16817634 0.11440004 ... 0.25396088 0.10606529 0.13573588]]

[[0.04002791 0.08294477 0.04524681 ... 0.22326142 0.03198272 0.04185161]
 [0.03661019 0.06661071 0.03627451 ..

In [22]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 16207. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2023-01-07 14:44:19,163 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-uormmsf5', purging
2023-01-07 14:44:19,163 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-07 14:44:19,163 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:43869': None}

In [ ]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

In [ ]:
automl_mgpu.to_cpu()

In [ ]:
mcpu_inf = automl_mgpu.predict(data)

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

In [ ]:
from joblib import dump, load
import time

pickle_file = './mgpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)